In [ ]:
import pandas as pd

In [ ]:
df3 = pd.read_csv('Bases de données/jeansville.csv')
df4 = pd.read_csv(/Bases de données/revenus.csv')
colonne_revenu= 'Niveau de Vie Commune'

# fusion des deux data frames
df5 = pd.merge(df3, df4[['colonne_revenu']], left_index=True, right_index=True, how='left')
print(df5)


In [ ]:
df5